In [1]:
#!pip list
import os
#print (os.environ["CUDA_VISIBLE_DEVICES"])
os.environ["CUDA_VISIBLE_DEVICES"] = '2,1'
print (os.environ["CUDA_VISIBLE_DEVICES"])
#import model file
import torch


2,1


In [2]:
import torch
torch.cuda.empty_cache()

In [3]:
import numpy as np
#import pandas as pd
from transformers import *
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from datasets import load_from_disk, load_metric
#import datasets
import pickle
#from utils.helper import read_py150k_code, read_file_to_string
import regex as re

In [4]:
#plbart init
#tokenizer = PLBartTokenizer.from_pretrained("uclanlp/plbart-base", src_lang="python", tgt_lang="python" )
#model = PLBartForConditionalGeneration.from_pretrained("uclanlp/plbart-base")
#model = PLBartForSequenceClassification.from_pretrained("uclanlp/plbart-base", num_labels=27)
import numpy as np

train_plbart_dataset = load_from_disk('datasets/plbart_train.hf')
test_plbart_dataset = load_from_disk('datasets/plbart_test.hf')
train_plbart_dataset.set_format(type="np", columns=["input_ids", "attention_mask", "labels"])
test_plbart_dataset.set_format(type="np", columns=["input_ids", "attention_mask", "labels"])

#codet5 init

tokenizer = PLBartTokenizer.from_pretrained("uclanlp/plbart-base", src_lang="python", tgt_lang="python")
#model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-base')
train_codet5_dataset = load_from_disk('datasets/codet5_train.hf')
test_codet5_dataset = load_from_disk('datasets/codet5_test.hf')

unk_id = tokenizer.convert_tokens_to_ids("<unk>")

def replaceNegative(example):
    #print (label['labels'])
    example['labels'] = (example['labels'] + 1)
    
    # remove duplicate eos token
    eos_mask = example["input_ids"] == 2
    if len(torch.unique_consecutive(torch.Tensor(eos_mask).sum(1))) > 1:
        invalid_idx = [idx for idx, mask in enumerate(eos_mask) if mask.sum() > 1]
        # print(invalid_idx)
        for idx in invalid_idx:
            invalid_token_idx = [idx for idx, mask in enumerate(eos_mask[idx]) if mask == 1]
            if len(invalid_token_idx) < 1: continue
            for invalid_token_pos in range(len(invalid_token_idx) - 1):
                # print(tokenizer.batch_decode([example["input_ids"][idx]]))
                example["input_ids"][idx][invalid_token_idx[invalid_token_pos]] = unk_id
                # print(tokenizer.batch_decode([example["input_ids"][idx]]))

    return example

train_plbart_dataset = train_plbart_dataset.map(replaceNegative, batched = True)
test_plbart_dataset = test_plbart_dataset.map(replaceNegative, batched = True)
train_plbart_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_plbart_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])



loading file https://huggingface.co/uclanlp/plbart-base/resolve/main/sentencepiece.bpe.model from cache at /soe/swade1/.cache/huggingface/transformers/e75bbc4eddf092f59fc4f715f4946a2e8401a821b9c5a4e3f60040dd8c1957ef.c65001d1986897f4ae6d41d2c49f0e1621d3518cab63e0ffa3005e5deb5aae40
loading file https://huggingface.co/uclanlp/plbart-base/resolve/main/tokenizer.json from cache at None
loading file https://huggingface.co/uclanlp/plbart-base/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/uclanlp/plbart-base/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/uclanlp/plbart-base/resolve/main/tokenizer_config.json from cache at None
loading configuration file https://huggingface.co/uclanlp/plbart-base/resolve/main/config.json from cache at /soe/swade1/.cache/huggingface/transformers/f6e5e7a0536acd8fb984d2eaf2bc23a6c6793d59fe0d72d104e56e1feacb98eb.8929a51af95d04be1b1d966435fd1ad4a48aca2f2953bc4e4a354b1f1dfd0b55
Model

  0%|          | 0/173 [00:00<?, ?ba/s]

  0%|          | 0/44 [00:00<?, ?ba/s]

In [5]:
tokenizer.convert_tokens_to_ids("<unk>")

3

In [6]:
sorted_dataset = test_plbart_dataset.sort('labels')
sorted_dataset['labels']

tensor([ 0,  0,  0,  ..., 26, 26, 26])

In [7]:
#sample and save data subset
torch.set_printoptions(threshold=10_000)
#sample_plbart = train_plbart_dataset.shuffle(seed=34).select(range(1000))
#sample_codet5 = train_codet5_dataset.shuffle(seed=34).select(range(1000))
#sample_plbart[0]
"""style_seperator = '[STY]'

sample = tokenizer(style_seperator, return_tensors="pt")
style_chars = []
for i in range (-1, 25):
    style_chars.append(f"<style{i}>")
print (style_chars)

#test_codet5_dataset.save_to_disk("datasets/plbart_sample_testing.hf")
#sample_codet5.save_to_disk("datasets/codet5_sample.hf")
example_python_phrase = "def maximum(a,b,c):NEW_LINE_INDENTreturn max([a,b,c])<style1>"
inputs = tokenizer(example_python_phrase, return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits    
    print(logits)"""

'style_seperator = \'[STY]\'\n\nsample = tokenizer(style_seperator, return_tensors="pt")\nstyle_chars = []\nfor i in range (-1, 25):\n    style_chars.append(f"<style{i}>")\nprint (style_chars)\n\n#test_codet5_dataset.save_to_disk("datasets/plbart_sample_testing.hf")\n#sample_codet5.save_to_disk("datasets/codet5_sample.hf")\nexample_python_phrase = "def maximum(a,b,c):NEW_LINE_INDENTreturn max([a,b,c])<style1>"\ninputs = tokenizer(example_python_phrase, return_tensors="pt")\nwith torch.no_grad():\n    logits = model(**inputs).logits    \n    print(logits)'

In [8]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, PLBartForSequenceClassification
model = PLBartForSequenceClassification.from_pretrained("uclanlp/plbart-base", num_labels=27)
#add classification layer

loading configuration file https://huggingface.co/uclanlp/plbart-base/resolve/main/config.json from cache at /soe/swade1/.cache/huggingface/transformers/f6e5e7a0536acd8fb984d2eaf2bc23a6c6793d59fe0d72d104e56e1feacb98eb.8929a51af95d04be1b1d966435fd1ad4a48aca2f2953bc4e4a354b1f1dfd0b55
Model config PLBartConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "architectures": [
    "PLBartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "dropout": 0.1,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
   

In [30]:
model.config

PLBartConfig {
  "_name_or_path": "uclanlp/plbart-base",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "architectures": [
    "PLBartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "dropout": 0.1,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19

In [9]:
print(torch.version.cuda)

11.3


In [11]:
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")
metric = load_metric("accuracy")
def compute_metrics(eval_pred): # this part prob wont work, parameter should be removed from trainer probably
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [12]:
from torch import nn
from transformers import Trainer


class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss (suppose one has 3 labels with different weights)
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 2.0, 3.0])) # calculate weights 
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [13]:
"""from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset["train"][100]

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.train()"""
#torch.cuda.set_device(2)
torch.cuda.current_device()
torch.cuda.current_device()


0

In [14]:

training_args = TrainingArguments(
    output_dir="./results/no_dataloader",
    learning_rate=1e-4, #2e-5, #
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    weight_decay=0.01
)
#debug_dataset = 
print (training_args.device)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_plbart_dataset,
    eval_dataset=test_plbart_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    #data_collator=data_collator,
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


cuda:0


The following columns in the training set don't have a corresponding argument in `PLBartForSequenceClassification.forward` and have been ignored: upper_camel_case, class_decorators_avg, lower_camel_case_ratio, snake_case_method, comment_avg_len, comment_total_len, ds_line_count, lambda, upper_camel_case_class, content, comment_density, class_parents_count, snake_case_var_ratio, func_async_count, lower_camel_case_method_ratio, lower_case_method, upper_camel_case_method_ratio, func_async_ratio, func_count, upper_case_class, file, upper_case_ratio, parse_error, lower_camel_case_var_ratio, lower_camel_case, issue_events, generators, lower_camel_case_class, func_decorators_count, forks, line_count, snake_case_class, ds_char_len_avg, other_case_method_ratio, other_case_class, upper_camel_case_ratio, upper_case_class_ratio, overridden_method, func_decorators_avg, ds_word_len_avg, ds_word_len_total, other_case_var_ratio, other_case_class_ratio, ds_char_len_total, lower_case_class_ratio, upper_

Step,Training Loss


RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/data/users/swade/miniconda3/envs/38/lib/python3.8/site-packages/torch/nn/parallel/parallel_apply.py", line 61, in _worker
    output = module(*input, **kwargs)
  File "/data/users/swade/miniconda3/envs/38/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1130, in _call_impl
    return forward_call(*input, **kwargs)
  File "/data/users/swade/miniconda3/envs/38/lib/python3.8/site-packages/transformers/models/plbart/modeling_plbart.py", line 1458, in forward
    outputs = self.model(
  File "/data/users/swade/miniconda3/envs/38/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1130, in _call_impl
    return forward_call(*input, **kwargs)
  File "/data/users/swade/miniconda3/envs/38/lib/python3.8/site-packages/transformers/models/plbart/modeling_plbart.py", line 1200, in forward
    decoder_outputs = self.decoder(
  File "/data/users/swade/miniconda3/envs